# Rubin et al. (2014) Head-Direction Validation

This notebook replicates the head-direction tuning analysis from Rubin, Yartsev & Ulanovsky (2014) using the new BatNavigationController. It simulates a crawling bat, records conjunctive place-cell activity, and compares head-direction tuning inside versus outside the primary place field.


In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

PROJECT_ROOT = Path(__file__).resolve().parents[1]
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

from hippocampus_core.controllers.bat_navigation_controller import (
    BatNavigationController,
    BatNavigationControllerConfig,
)
from hippocampus_core.env import Agent, Environment



In [ ]:
def run_simulation(
    duration_seconds: float = 300.0,
    dt: float = 0.05,
    num_place_cells: int = 60,
    seed: int = 7,
):
    env = Environment(width=1.0, height=1.0)
    config = BatNavigationControllerConfig(
        num_place_cells=num_place_cells,
        hd_num_neurons=72,
        grid_size=(16, 16),
        calibration_interval=250,
        integration_window=480.0,
    )
    controller = BatNavigationController(env, config=config, rng=np.random.default_rng(seed))
    agent = Agent(env, random_state=np.random.default_rng(seed + 1), track_heading=True)

    num_steps = int(duration_seconds / dt)
    obs_history = np.zeros((num_steps, 3), dtype=float)
    rate_history = np.zeros((num_steps, num_place_cells), dtype=float)

    for idx in range(num_steps):
        obs = agent.step(dt, include_theta=True)
        controller.step(obs, dt)
        obs_history[idx] = obs
        rate_history[idx] = controller.last_rates

    return obs_history, rate_history, controller.place_cells



In [ ]:
def rayleigh_vector(theta: np.ndarray, weights: np.ndarray) -> float:
    theta = np.asarray(theta)
    weights = np.asarray(weights)
    vector = np.sum(weights * np.exp(1j * theta))
    return np.abs(vector) / np.sum(weights)


def compute_tuning(obs_history, rate_history, place_cell_centers, sigma, cell_index=None):
    if cell_index is None:
        cell_index = int(np.argmax(rate_history.mean(axis=0)))

    positions = obs_history[:, :2]
    headings = obs_history[:, 2]
    rates = rate_history[:, cell_index]
    center = place_cell_centers[cell_index, :2]

    distance = np.linalg.norm(positions - center, axis=1)
    in_mask = distance <= sigma
    out_mask = ~in_mask

    heading_bins = np.linspace(-np.pi, np.pi, 25)
    def tuning_curve(mask):
        counts, _ = np.histogram(headings[mask], bins=heading_bins, weights=rates[mask])
        sample_counts, _ = np.histogram(headings[mask], bins=heading_bins)
        with np.errstate(divide="ignore", invalid="ignore"):
            curve = np.where(sample_counts > 0, counts / sample_counts, 0.0)
        centers = 0.5 * (heading_bins[1:] + heading_bins[:-1])
        return centers, curve

    centers, in_curve = tuning_curve(in_mask)
    _, out_curve = tuning_curve(out_mask)

    return {
        "cell_index": cell_index,
        "center": center,
        "in_mask": in_mask,
        "out_mask": out_mask,
        "rayleigh_in": rayleigh_vector(headings[in_mask], rates[in_mask] + 1e-6),
        "rayleigh_out": rayleigh_vector(headings[out_mask], rates[out_mask] + 1e-6),
        "curve_centers": centers,
        "in_curve": in_curve,
        "out_curve": out_curve,
        "positions": positions,
        "headings": headings,
        "rates": rates,
    }



In [ ]:
obs_history, rate_history, place_cells = run_simulation()
analysis = compute_tuning(
    obs_history,
    rate_history,
    place_cells.get_positions(),
    sigma=BatNavigationControllerConfig().sigma,
)
analysis


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
centers = analysis["curve_centers"]
axes[0].plot(np.degrees(centers), analysis["in_curve"], label="in-field")
axes[0].plot(np.degrees(centers), analysis["out_curve"], label="out-of-field")
axes[0].set_xlabel("Head direction (deg)")
axes[0].set_ylabel("Mean rate (Hz)")
axes[0].set_title(
    f"Rayleigh | in={analysis['rayleigh_in']:.2f}, out={analysis['rayleigh_out']:.2f}"
)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

positions = analysis["positions"]
axes[1].scatter(positions[:, 0], positions[:, 1], c=analysis["rates"], cmap="viridis", s=5)
axes[1].scatter(
    analysis["center"][0], analysis["center"][1], marker="*", s=150, c="red", label="cell center"
)
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")
axes[1].set_title("Place cell firing intensity")
axes[1].legend()
axes[1].set_aspect("equal")
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


The default configuration already reproduces the qualitative findings of Rubin et al. (2014): head-direction tuning is significant inside the preferred place field and remains robust outside it. Adjust the simulation duration, number of cells, or thresholds in the helper functions above to explore different recording conditions.
